In [1]:
# Needed libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.key_input import KeyInput
from selenium.webdriver.common.actions.pointer_input import PointerInput

# System requiremenmts 
import time
import random
import datetime as dt

# Data Management 
import pandas as pd

# Web scrapping functions 
import src.web_minning as wm

In [2]:
# Parameters 

CITY = "cdmx"
TAGS_LIST = wm.city_hashtags(CITY)
MAX_DEPTH = 5 # Iteration loop over each initial video
#MAX_SCROLLING = #

RAW_DATA = "./data/raw/"
ROOT_FILE = "root_links.csv"
DAY_FILE = f"day_file_{CITY}_{pd.to_datetime(dt.datetime.today()).strftime('%Y-%m-%d')}.csv"

In [3]:
results = []


for ht in TAGS_LIST:


    try:
        # Invoke driver object 

        driver = webdriver.Chrome()
        wait = WebDriverWait(driver, 15)
        actions = ActionChains(driver)
        keyboard = KeyInput("keyboard")
        actions_b = ActionBuilder(driver, keyboard=keyboard)


        # Open Main page
        driver.get("https://www.tiktok.com")


        # Look for search icon 
        search_icon = wait.until(EC.presence_of_element_located((
            By.XPATH, "//*[@role='searchbox']"
        )))

        # Put mose in search bar 
        search_icon.click()

        # Input labels 
        actions.send_keys(ht)
        time.sleep(random.uniform(0.05, 0.15))
        actions.perform()

        # Enter to look 
        actions.send_keys("\n")
        actions.perform()

        time.sleep(random.uniform(1.5, 4.5))


        # Getting all inmediate results 
        items = wm.scroll_until_no_new_items(driver)
        wm.scroll_through_elements(driver, items)
        items = driver.find_elements(By.CSS_SELECTOR, "div[id^='grid-item-container-']")

        # Get data from results

        for container in items:
            try:
                href = container.find_element(
                    By.CSS_SELECTOR, "a[href*='/video/']"
                ).get_attribute("href")

                description = container.find_element(
                    By.CSS_SELECTOR, "[data-e2e='new-desc-span']"
                ).text.strip()

                date = container.find_element(
                    By.CSS_SELECTOR, "div[class*='DivTimeTag']"
                ).text.strip()

                results.append({
                    "href": href,
                    "description": description,
                    "date": date,
                    "hashtag_label": ht
                })

            except Exception:
                continue

        print(f"Links search for {ht} ended.")

        driver.quit()

    except:
        driver.quit()



Links search for #dondeircdmx ended.
Links search for #lugarescdmx ended.
Links search for #places #cdmx ended.
Links search for #placescdmx ended.
Links search for #turismocdmx ended.
Links search for #lugaresbonitos #cdmx ended.
Links search for #lugaressecretos #cdmx ended.
Links search for #lugaresimperdibles #cdmx ended.
Links search for #lugaresrecomendados #cdmx ended.
Links search for #experiencias #cdmx ended.
Links search for #experienciasunicas #cdmx ended.
Links search for #lugaresbonitoscdmx ended.
Links search for #lugaressecretoscdmx ended.
Links search for #findesemana #cdmx ended.
Links search for #findesemanacdmx ended.
Links search for #planesdefin #cdmx ended.
Links search for #planperfecto  #cdmx ended.
Links search for #escapadacdmx ended.
Links search for #brunch #cdmx ended.
Links search for #cafes #cdmx ended.
Links search for #restaurantes #cdmx ended.
Links search for #brunchcdmx ended.
Links search for #cafescdmx ended.
Links search for #comidacdmx ended.
Li

In [4]:
df_raw = pd.DataFrame(results)
df_raw['level'] = 0

In [5]:
df_raw.to_csv(RAW_DATA + ROOT_FILE, index = False)

In [6]:
root_list = df_raw.href.drop_duplicates().to_list()

desc_results = []

for h in root_list:

    try:

        driver = webdriver.Chrome()
        wait = WebDriverWait(driver, 15)
        actions = ActionChains(driver)
        keyboard = KeyInput("keyboard")
        actions_b = ActionBuilder(driver, keyboard=keyboard)

        driver.get(h)

        time.sleep(random.uniform(2.5,3.5))

        more_buton = driver.find_element(
            By.CSS_SELECTOR,
            "button[class*='ButtonExpand']"
        )

        more_buton.click()

        time.sleep(random.uniform(2.5,3.5))


        # Date Extraction 
        try:
            raw = driver.find_element(
            By.CSS_SELECTOR,
            "span[class*='StyledTUXText']"
            ).text
            date_text = raw.replace("·","").replace(" ","")
        except Exception:
            date_text = ""

        time.sleep(random.uniform(2.5,3.5))

        # Description extraction
        try:
            raw = driver.find_element(
            By.CSS_SELECTOR,
            "div[class*='DivCustomTDKContainer']"
            ).text

            ia_disclaimer = "Esta información se generó por IA y puede presentar resultados que no son relevantes. No representa las opiniones o consejos de TikTok. Si tienes alguna duda, envíanosla a través de: Comentarios y ayuda: TikTok"

            desc_text = raw.replace(ia_disclaimer,"")

        except Exception:
            desc_text = ""

        time.sleep(random.uniform(0.5,1.5))

        desc_results.append({
                    "href": h,
                    "long_description": desc_text,
                    "date_long": date_text
                                            })


        driver.quit()

    except:
        driver.quit()

In [7]:
df_raw_des = pd.DataFrame(desc_results)

In [8]:
df_raw_des.to_csv(RAW_DATA + DAY_FILE, index = False)

In [9]:
len(df_raw_des)

755

In [10]:
len(df_raw)

1478